In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pymongo
import requests

In [2]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
sido = starbucks['sido']
sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})

시도 이름을 입력받아 시도 이름에 해당되는 시도 코드를 sido 컬렉션에서 얻어온 후 크롤링하기

In [3]:
for s in sidoList:
    print(s['sido_cd'], end=' ')

01 08 02 03 04 05 06 07 09 10 11 12 13 14 15 16 17 

In [4]:
sidoName = input('구군 코드를 검색하려는 시도 이름을 입력하세요(예: 서울): ')
# sido 컬렉션에서 입력받은 시도 이름에 해당되는 도큐먼트 1건을 얻어온다.
try:
    sido_cd = sido.find({'sido_nm': sidoName}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1}).next()
    # print(sido_cd['sido_cd'])
    request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', 
                data={
                    'sido_cd': sido_cd['sido_cd']
                }).json()
    # print(request)
    
    gugunList = []
    for r in request['list']:
        # 세종은 구분되는 gugun_nm이 없으므로 None이 리턴된다. 세종 None 처리
        if r.get('gugun_nm') is None:
            r['gugun_nm'] = '세부지역없음'
        d = {'gugun_cd': r['gugun_cd'].strip(), 'gugun_nm': r.get('gugun_nm').strip()}
        # print(d)
        gugunList.append(d)
    print(gugunList)
    
    gugun = starbucks['gugun'] # db.createCollection('gugun')
    gugun.delete_many({})
    for s in gugunList:
        gugun.insert_one(s)
except:
    print(f'{sidoName}는(은) 잘못된 시도 이름입니다.')

구군코드를 검색하려는 시도 이름을 입력하세요(예: 서울): 경기
[{'gugun_cd': '0801', 'gugun_nm': '가평군'}, {'gugun_cd': '0832', 'gugun_nm': '고양시 덕양구'}, {'gugun_cd': '0802', 'gugun_nm': '고양시 일산동구'}, {'gugun_cd': '0836', 'gugun_nm': '고양시 일산서구'}, {'gugun_cd': '0803', 'gugun_nm': '과천시'}, {'gugun_cd': '0804', 'gugun_nm': '광명시'}, {'gugun_cd': '0805', 'gugun_nm': '광주시'}, {'gugun_cd': '0806', 'gugun_nm': '구리시'}, {'gugun_cd': '0807', 'gugun_nm': '군포시'}, {'gugun_cd': '0837', 'gugun_nm': '김포시'}, {'gugun_cd': '0808', 'gugun_nm': '남양주시'}, {'gugun_cd': '0844', 'gugun_nm': '동두천시'}, {'gugun_cd': '0838', 'gugun_nm': '부천시'}, {'gugun_cd': '0811', 'gugun_nm': '성남시 분당구'}, {'gugun_cd': '0833', 'gugun_nm': '성남시 수정구'}, {'gugun_cd': '0812', 'gugun_nm': '성남시 중원구'}, {'gugun_cd': '0813', 'gugun_nm': '수원시 권선구'}, {'gugun_cd': '0839', 'gugun_nm': '수원시 영통구'}, {'gugun_cd': '0814', 'gugun_nm': '수원시 장안구'}, {'gugun_cd': '0815', 'gugun_nm': '수원시 팔달구'}, {'gugun_cd': '0816', 'gugun_nm': '시흥시'}, {'gugun_cd': '0817', 'gugun_nm': '안산시 단원구'}, {'gugun_cd': 

In [5]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
gugun = starbucks['gugun']
gugunList = gugun.find({}, {'_id': 0, 'gugun_cd': 1, 'gugun_nm': 1})
for s in gugunList:
    print(s)

{'gugun_cd': '0801', 'gugun_nm': '가평군'}
{'gugun_cd': '0832', 'gugun_nm': '고양시 덕양구'}
{'gugun_cd': '0802', 'gugun_nm': '고양시 일산동구'}
{'gugun_cd': '0836', 'gugun_nm': '고양시 일산서구'}
{'gugun_cd': '0803', 'gugun_nm': '과천시'}
{'gugun_cd': '0804', 'gugun_nm': '광명시'}
{'gugun_cd': '0805', 'gugun_nm': '광주시'}
{'gugun_cd': '0806', 'gugun_nm': '구리시'}
{'gugun_cd': '0807', 'gugun_nm': '군포시'}
{'gugun_cd': '0837', 'gugun_nm': '김포시'}
{'gugun_cd': '0808', 'gugun_nm': '남양주시'}
{'gugun_cd': '0844', 'gugun_nm': '동두천시'}
{'gugun_cd': '0838', 'gugun_nm': '부천시'}
{'gugun_cd': '0811', 'gugun_nm': '성남시 분당구'}
{'gugun_cd': '0833', 'gugun_nm': '성남시 수정구'}
{'gugun_cd': '0812', 'gugun_nm': '성남시 중원구'}
{'gugun_cd': '0813', 'gugun_nm': '수원시 권선구'}
{'gugun_cd': '0839', 'gugun_nm': '수원시 영통구'}
{'gugun_cd': '0814', 'gugun_nm': '수원시 장안구'}
{'gugun_cd': '0815', 'gugun_nm': '수원시 팔달구'}
{'gugun_cd': '0816', 'gugun_nm': '시흥시'}
{'gugun_cd': '0817', 'gugun_nm': '안산시 단원구'}
{'gugun_cd': '0840', 'gugun_nm': '안산시 상록구'}
{'gugun_cd': '0818', 'gugun_

시도 이름에 따른 전체 구군 정보를 gugunAll 컬렉션에 도큐먼트로 저장하기

In [6]:
# starbucks 데이터베이스의 sido 컬렉션에 저장된 도큐먼트의 개수만큼 반복하며 모든 구군 정보를 gugunAll 컬렉션에 도큐먼트로 저장한다.
gugunList = []
sidoList = sido.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1})
for s in sidoList:
    request = requests.post('https://www.starbucks.co.kr/store/getGugunList.do', 
                data={
                    'sido_cd': s['sido_cd']
                }).json()
    for r in request['list']:
        if r.get('gugun_nm') is None:
            r['gugun_nm'] = '세부지역없음'
        d = {'sido_cd': s['sido_cd'].strip(), 'sido_nm': s.get('sido_nm').strip(), 'gugun_cd': r['gugun_cd'].strip(), 
             'gugun_nm': r.get('gugun_nm').strip()}
        # print(d)
        gugunList.append(d)
    # print(gugunList)
# ===== for

gugunAll = starbucks['gugunAll'] # db.createCollection('gugunAll')
gugunAll.delete_many({})
for s in gugunList:
    gugunAll.insert_one(s)

In [7]:
mongoConnection = pymongo.MongoClient('mongodb://localhost:27017')
starbucks = mongoConnection['starbucks']
gugunAll = starbucks['gugunAll']
# gugunList = gugunAll.find({}, {'_id': 0, 'sido_cd': 1, 'sido_nm': 1, 'gugun_cd': 1, 'gugun_nm': 1})
gugunList = gugunAll.find({}, {'_id': 0})
for s in gugunList:
    print(s)

{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0101', 'gugun_nm': '강남구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0120', 'gugun_nm': '강동구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0102', 'gugun_nm': '강북구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0103', 'gugun_nm': '강서구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0104', 'gugun_nm': '관악구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0105', 'gugun_nm': '광진구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0121', 'gugun_nm': '구로구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0106', 'gugun_nm': '금천구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0107', 'gugun_nm': '노원구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0108', 'gugun_nm': '도봉구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0122', 'gugun_nm': '동대문구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0109', 'gugun_nm': '동작구'}
{'sido_cd': '01', 'sido_nm': '서울', 'gugun_cd': '0110', 'gugun_nm': '마포구'}
{'sido_cd': '01', 'sido_nm': '서울', 'g